# CDP Connector with lazy frames using Polars

## Setup

In [2]:
import pandas as pd
df_main =  pd.read_csv('ACC_minute_data_with_indicators.csv')


## Connect to CDP source using core and refresh token

In [41]:
from salesforcecdpconnector.connection import SalesforceCDPConnection



cur = conn.cursor()



### Get Dataframes

#### Pandas data frame

In [40]:
data_frame = conn.get_pandas_dataframe('SELECT bottles_sold__c, category_name__c, cdp_sys_SourceVersion__c, DataSource__c, DataSourceObject__c, id__c, InternalOrganization__c, item_description__c, item_number__c, store_number__c FROM LiquorSales__dll LIMIT 10000000')

data_frame

Error: Token Renewal failed with code 400

#### Polars lazy frame

## Merging csv to master file

In [2]:
import os
import pandas as pd
df_main = pd.read_csv('ACC_minute_data_with_indicators.csv')
df_main['row_number'] = df_main.reset_index().index
for subdir, dirs, files in os.walk('./'):
    for file in files:
      if (file == 'ACC_minute_data_with_indicators.csv' or file == 'master_file.csv' or file == 'master_file_small.csv'):
        continue

      res=file.split('.')
      if (res[1] == 'csv'):
        print(file + ' ... merging ...')
        df_cur = pd.read_csv(file)
        df_cur['row_number'] = df_cur.reset_index().index
        df_main = df_main.append(df_cur)
        print('current_size = ' + str(df_main.shape[0]))

df_main.to_csv('master_file_extreme_small.csv')


ADANIGREEN_minute_data_with_indicators.csv ... merging ...
current_size = 1031105
ADANIPORTS_minute_data_with_indicators.csv ... merging ...
current_size = 1702138
ADANIENT_minute_data_with_indicators.csv ... merging ...
current_size = 2362614
APOLLOHOSP_minute_data_with_indicators.csv ... merging ...
current_size = 3033648
AMBUJACEM_minute_data_with_indicators.csv ... merging ...
current_size = 3699434


## Testing performance of grouping

In [2]:
import os
for subdir, dirs, files in os.walk('./'):
    for file in files:
        print(file)

test.rs
master_file_extreme_small.csv
.DS_Store
ADANIGREEN_minute_data_with_indicators.csv
testFile.csv
ADANIPORTS_minute_data_with_indicators.csv
ADANIENT_minute_data_with_indicators.csv
cdp.ipynb
APOLLOHOSP_minute_data_with_indicators.csv
AMBUJACEM_minute_data_with_indicators.csv
ACC_minute_data_with_indicators.csv


In [1]:
import pandas as pd
df_main = pd.read_csv('master_file_small.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'master_file_small.csv'

In [43]:

filtered_n = df_main[df_main['volume'] >= 200]
groups_n = filtered_n.groupby(['date']).agg({'high':'mean', 'low':'mean', 'close':'mean'})
groups_n
#groups_n = df_main.groupby(['date'])['high'].agg('mean')
#df_main.shape[0]

,high,low,close
date,,,
2015-02-02 10:18:00+05:30,457.215714,456.770000,456.922857
2015-02-02 10:19:00+05:30,542.683846,542.114615,542.337692
2015-02-02 10:20:00+05:30,373.343077,372.725385,372.961538
2015-02-02 10:21:00+05:30,444.650769,444.045385,444.215385
2015-02-02 10:22:00+05:30,552.745000,552.080000,552.565714
...,...,...,...
2022-10-25 12:21:00+05:30,3697.150000,3694.050000,3695.900000
2022-10-25 12:22:00+05:30,3694.100000,3690.100000,3692.400000
2022-10-25 12:23:00+05:30,3695.300000,3691.250000,3694.950000


In [29]:
#df_main.apply(lambda row: row[3]-row[2], axis=1)
df_main['result2'] = df_main['high'] + df_main['low']

In [31]:
import polars as pl
df_polars = pl.from_pandas(df_main)

In [ ]:

#%timeit df_polars['result2'] - df_polars.apply(lambda row: row[3]+row[2])
df_polars.with_columns([
    (pl.col('high') - pl.col('low')).alias('result')
])

In [41]:
q = (
    df_polars
    .lazy()
    .filter(pl.col('volume') >= 200)
    .groupby(by='date')
    .agg(pl.col('high').mean().alias('high(mean)'))
)
q.collect()

date,high(mean)
str,f64
"""2015-11-17 10:...",497.207692
"""2016-04-06 13:...",536.6225
"""2017-04-26 10:...",805.669412
"""2017-10-23 15:...",822.840625
"""2018-02-22 09:...",780.94375
"""2018-12-05 09:...",1792.8325
"""2021-03-02 11:...",1375.45
"""2022-09-05 15:...",1961.405263
"""2015-02-25 14:...",556.602143


In [18]:
df_polars.apply(lambda row: row[3]-row[2])

apply
f64
1.0
0.3
0.7
0.7
0.05
1.1
0.65
0.0
2.4


In [10]:
from salesforcecdpconnector.connection import SalesforceCDPConnection



In [14]:
data_frame3 = conn.get_pandas_dataframe('SELECT bottles_sold__c, category_name__c, cdp_sys_SourceVersion__c, DataSource__c, DataSourceObject__c, id__c, InternalOrganization__c, item_description__c, item_number__c, store_number__c FROM LiquorSales__dll LIMIT 1000')

data_frame3

,bottles_sold__c,category_name__c,cdp_sys_SourceVersion__c,DataSource__c,DataSourceObject__c,id__c,InternalOrganization__c,item_description__c,item_number__c,store_number__c
0,24.000000000000000000,BLENDED WHISKIES,None,LiquorSales,s3://ed-cdp-257c67b4-7123-11ed-a1eb-0242ac1200...,1.000000000000000000,None,Hawkeye Blend Whiskey,24156.000000000000000000,4465.000000000000000000
1,36.000000000000000000,SPICED RUM,None,LiquorSales,s3://ed-cdp-257c67b4-7123-11ed-a1eb-0242ac1200...,4.000000000000000000,None,Captain Morgan Spiced Rum,43337.000000000000000000,3842.000000000000000000
2,72.000000000000000000,WHISKEY LIQUEUR,None,LiquorSales,s3://ed-cdp-257c67b4-7123-11ed-a1eb-0242ac1200...,8.000000000000000000,None,Fireball Cinnamon Whiskey,64867.000000000000000000,4312.000000000000000000
3,24.000000000000000000,VODKA 80 PROOF,None,LiquorSales,s3://ed-cdp-257c67b4-7123-11ed-a1eb-0242ac1200...,14.000000000000000000,None,Titos Vodka,38176.000000000000000000,3562.000000000000000000
4,48.000000000000000000,VODKA 80 PROOF,None,LiquorSales,s3://ed-cdp-257c67b4-7123-11ed-a1eb-0242ac1200...,16.000000000000000000,None,Mccormick Vodka Pet,36904.000000000000000000,3820.000000000000000000
...,...,...,...,...,...,...,...,...,...,...
995,48.000000000000000000,VODKA 80 PROOF,None,LiquorSales,s3://ed-cdp-257c67b4-7123-11ed-a1eb-0242ac1200...,1948.000000000000000000,None,Mccormick Vodka,36903.000000000000000000,4528.000000000000000000
996,108.000000000000000000,CANADIAN WHISKIES,None,LiquorSales,s3://ed-cdp-257c67b4-7123-11ed-a1eb-0242ac1200...,1951.000000000000000000,None,Black Velvet,11788.000000000000000000,3385.000000000000000000
997,24.000000000000000000,SPICED RUM,None,LiquorSales,s3://ed-cdp-257c67b4-7123-11ed-a1eb-0242ac1200...,1952.000000000000000000,None,Captain Morgan Spiced Rum,43334.000000000000000000,4421.000000000000000000
998,24.000000000000000000,CANADIAN WHISKIES,None,LiquorSales,s3://ed-cdp-257c67b4-7123-11ed-a1eb-0242ac1200...,1953.000000000000000000,None,Black Velvet Toasted Caramel,10550.000000000000000000,2560.000000000000000000


In [15]:
data_frame2 = conn.get_polars_dataframe('SELECT bottles_sold__c, category_name__c, cdp_sys_SourceVersion__c, DataSource__c, DataSourceObject__c, id__c, InternalOrganization__c, item_description__c, item_number__c, store_number__c FROM LiquorSales__dll LIMIT 1000')

data_frame2

bottles_sold__c,category_name__c,cdp_sys_SourceVersion__c,DataSource__c,DataSourceObject__c,id__c,InternalOrganization__c,item_description__c,item_number__c,store_number__c
f64,str,str,str,str,f64,str,str,f64,f64
24.0,"""BLENDED WHISKI...",null,"""LiquorSales""","""s3://ed-cdp-25...",1.0,null,"""Hawkeye Blend ...",24156.0,4465.0
36.0,"""SPICED RUM""",null,"""LiquorSales""","""s3://ed-cdp-25...",4.0,null,"""Captain Morgan...",43337.0,3842.0
72.0,"""WHISKEY LIQUEU...",null,"""LiquorSales""","""s3://ed-cdp-25...",8.0,null,"""Fireball Cinna...",64867.0,4312.0
24.0,"""VODKA 80 PROOF...",null,"""LiquorSales""","""s3://ed-cdp-25...",14.0,null,"""Titos Vodka""",38176.0,3562.0
48.0,"""VODKA 80 PROOF...",null,"""LiquorSales""","""s3://ed-cdp-25...",16.0,null,"""Mccormick Vodk...",36904.0,3820.0
24.0,"""VODKA 80 PROOF...",null,"""LiquorSales""","""s3://ed-cdp-25...",17.0,null,"""Mccormick Vodk...",36904.0,4590.0
48.0,"""BLENDED WHISKI...",null,"""LiquorSales""","""s3://ed-cdp-25...",19.0,null,"""Seagrams 7 Cro...",25607.0,2633.0
24.0,"""IMPORTED SCHNA...",null,"""LiquorSales""","""s3://ed-cdp-25...",20.0,null,"""Rumple Minze P...",69947.0,5102.0
60.0,"""IMPORTED VODKA...",null,"""LiquorSales""","""s3://ed-cdp-25...",23.0,null,"""Absolut Citron...",34029.0,4829.0


In [4]:
import pandas as pd
test_df = pd.read_csv('testFile2.csv')
test_df

,index,cat,open
0,0,1,408570
1,1,0,469494
2,2,4,209883
3,3,3,180007
4,4,4,61813
...,...,...,...
199999995,199999995,3,58397
199999996,199999996,4,74796
199999997,199999997,0,132541
199999998,199999998,3,263675


In [5]:
test_filter = test_df[test_df['open'] >= 40000]
test_group = test_filter.groupby(['cat']).agg({'open':'mean'})
test_group

,open
cat,
0,520007.438380
1,520338.031112
2,519841.798749
3,519976.329641
4,519925.805426


In [6]:
import polars as pl
test_polars = pl.from_pandas(test_df)

In [7]:
q = (
    test_polars
    .lazy()
    .filter(pl.col('open') >= 40000)
    .groupby(by='cat')
    .agg(pl.col('open').mean().alias('open(mean)'))
)
q.collect()

cat,open(mean)
i64,f64
0,520007.43838
2,519841.798749
4,519925.805426
1,520338.031112
3,519976.329641
